In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import datetime

In [2]:
url = '/Users/skydivedavis/Downloads/seasonal_mta_data_01.csv'
df = pd.read_csv(url, header = 0, sep=',')
df.columns = map(str.lower, df.columns) # - make headers lowercase
df.columns = df.columns.str.strip().str.lower()

In [3]:
print (df.head(5))


   unnamed: 0   c/a  unit       scp station linename division        date  \
0           0  A002  R051  02-00-00   59 ST   NQR456      BMT  02/27/2016   
1           1  A002  R051  02-00-00   59 ST   NQR456      BMT  02/27/2016   
2           2  A002  R051  02-00-00   59 ST   NQR456      BMT  02/27/2016   
3           3  A002  R051  02-00-00   59 ST   NQR456      BMT  02/27/2016   
4           4  A002  R051  02-00-00   59 ST   NQR456      BMT  02/27/2016   

       time     desc  entries    exits  
0  03:00:00  REGULAR  5562172  1877574  
1  07:00:00  REGULAR  5562184  1877586  
2  11:00:00  REGULAR  5562267  1877691  
3  15:00:00  REGULAR  5562516  1877745  
4  19:00:00  REGULAR  5562920  1877821  


In [4]:
#Convert a tuple into a string for indexing purposes. This is an unfortunate kludge that was necessary due to having
#trouble using a tuple as an index
def string_from_tuple (tup):
    return tup[0]+','+tup[1]+','+str(tup[2])


df['ttime'] = pd.to_datetime(df['time'])
df['tdate'] = pd.to_datetime(df['date'])
df['datetime'] = df.apply (lambda x: datetime.datetime (x['tdate'].year, x['tdate'].month, x['tdate'].day,
                                                       x['ttime'].hour), axis=1)
df.drop (['ttime','tdate'], inplace=True, axis=1)

df['exit_increments'] = df['exits']
df['entry_increments'] = df['entries']
df['previous_entry_number'] = df['entries']
df['time_block'] = df['entries']
df['previous_datetime'] = df['datetime']

In [5]:
print (df)

          unnamed: 0    c/a  unit       scp        station linename division  \
0                  0   A002  R051  02-00-00          59 ST   NQR456      BMT   
1                  1   A002  R051  02-00-00          59 ST   NQR456      BMT   
2                  2   A002  R051  02-00-00          59 ST   NQR456      BMT   
3                  3   A002  R051  02-00-00          59 ST   NQR456      BMT   
4                  4   A002  R051  02-00-00          59 ST   NQR456      BMT   
5                  5   A002  R051  02-00-00          59 ST   NQR456      BMT   
6                  6   A002  R051  02-00-00          59 ST   NQR456      BMT   
7                  7   A002  R051  02-00-00          59 ST   NQR456      BMT   
8                  8   A002  R051  02-00-00          59 ST   NQR456      BMT   
9                  9   A002  R051  02-00-00          59 ST   NQR456      BMT   
10                10   A002  R051  02-00-00          59 ST   NQR456      BMT   
11                11   A002  R051  02-00

In [6]:
#This creates the dataframe that will be used to store entries for a given station/control area pair for hours
#of the day
tuple1 = ('59 ST', 'A002', 1)

#Convert a tuple into a string for indexing purposes. This is an unfortunate kludge that was necessary due to having
#trouble using a tuple as an index
def string_from_tuple (tup):
    return tup[0]+','+tup[1]+','+str(tup[2])

summary_data = pd.DataFrame(
    data={'c_a_station_time_block': [tuple1],
          'c_a_station': [('59 ST', 'A002')],
          'time_block': [1],
          'entries': [0]},
    index=[string_from_tuple (tuple1)],
    columns=['c_a_station_time_block', 'c_a_station', 'time_block', 'entries'])

#Tabulate the number of entries over a particular four hour block at a particular station/control area pair
def tabulate_entries (ix, data_frame):
    station = data_frame.at [ix, 'station']
    control_area = data_frame.at [ix, 'c/a']
    time_block = data_frame.at [ix, 'time_block']
    entries = data_frame.at [ix, 'entry_increments']
    c_a_station = (station, control_area)
    c_a_station_time_block = (station, control_area, time_block)
    summary_index = string_from_tuple (c_a_station_time_block)
    try:
        existing_entries = summary_data.at [summary_index, 'entries']
        summary_data.at [summary_index, 'entries'] = entries + existing_entries
    except KeyError:
        summary_data.loc [summary_index] = [c_a_station_time_block, c_a_station, time_block, entries]

#Converts a datetime to an integer code representing the four hour block in the week, because
#MTA generally uses four hour intervals. 12am-2am Saturday = 0, 2am-6am Saturday = 1, and so on up to
#6pm-10pm Friday = 41, 10pm-midnight Friday = 42
def convert_datetime_to_four_hour_block (dt):
    day_of_week = dt.weekday()
    day_of_week += 2
    if day_of_week >= 7:
        day_of_week -= 7
    hour = round((dt.hour+.01)/4)
    result = int((6*day_of_week) + hour)
    return result

def max_minus_min_exits (x):
    first_iteration = True
    previous_exits = 0
    for index, row in x.iteritems():
        if first_iteration:
            first_iteration = False
            previous_exits = row
            df.at [index, 'exit_increments'] = -1
        else:
            df.at [index, 'exit_increments'] = row - previous_exits
            previous_exits = row
    result = x.max()-x.min()
    return result
    
def max_minus_min_entries (x):
    first_iteration = True
    previous_entries = 0
    previous_datetime = datetime.datetime (1980,1,1)
    for index, row in x.iteritems():
        if first_iteration:
            first_iteration = False
            previous_entries = row
            df.at [index, 'previous_datetime'] = previous_datetime
            previous_datetime = df.at [index, 'datetime']
            df.at [index, 'entry_increments'] = -1
            df.at [index, 'previous_entry_number'] = -1
            df.at [index, 'time_block'] = -1
        else:
            df.at [index, 'previous_datetime'] = previous_datetime
            previous_datetime = df.at [index, 'datetime']
            entry_count = row - previous_entries
            df.at [index, 'entry_increments'] = entry_count
            df.at [index, 'previous_entry_number'] = previous_entries
            df.at [index, 'time_block'] = convert_datetime_to_four_hour_block (previous_datetime)
            if (entry_count >0) and (entry_count < 5000):
                tabulate_entries (index, df)
            previous_entries = row
    result = x.max()-x.min()
    return result
    
dfgb_exit_group = df.groupby (['station', 'c/a', 'unit', 'scp']) ['exits'].apply (max_minus_min_exits)
dfgb_entry_group = df.groupby (['station', 'c/a', 'unit', 'scp']) ['entries'].apply (max_minus_min_entries)
dfgb2 = dfgb_exit_group.groupby ('station')
dfgb3 = dfgb2.apply (lambda x:x.sum())
dfgb4 = dfgb3.sort_values (0, False)
#print (dfgb4)
#print (dfgb4.sum())
#print(len(dfgb4))
dfgb4.tail(360).sum()
dfgb=dfgb_exit_group.sort_values (0, False)
print (dfgb.sum())
print (len(dfgb))
print (dfgb)
dfgb.tail(4875).sum()

157781935560
5141
station          c/a    unit  scp     
BOWLING GREEN    R201   R041  00-00-00    2145171687
61 ST WOODSIDE   R523   R147  00-00-04    2110118363
LORIMER ST       J005   R353  00-00-01    2097169634
34 ST-HERALD SQ  A025   R023  01-03-02    2049820042
                 N506   R022  00-05-04    2041457012
7 AV             N539   R288  00-03-01    2029873651
47-50 STS ROCK   N501   R020  01-03-02    2025547066
CANAL ST         A066   R118  00-00-02    2023163078
1 AV             H008   R248  01-00-00    2013192114
ELMHURST AV      N325A  R218  00-05-01    1973057511
                              00-06-01    1944272744
104 ST           J034   R007  00-00-01    1906327580
HIGH ST          N100   R252  00-00-01    1899462396
28 ST            R228   R143  00-00-01    1880724421
182-183 STS      N215   R237  00-00-02    1838798762
190 ST           N006A  R280  00-00-00    1836854532
167 ST           N207   R104  00-00-00    1822962385
34 ST-HERALD SQ  A025   R023  01-03-01    

6653005617

In [7]:
import matplotlib.pyplot as plt
import matplotlib

from matplotlib.pyplot import figure
figure(num=None, figsize=(14, 8), dpi=80, facecolor='w', edgecolor='k')

legends = []

# This function reorders a sequence of x values that may not be in the right order for plotting
# For now this is not implemented in a very efficient way
# This function takes as a parameter a station and control area pair
def do_plotting (station, control_area):
    xsa = []
    ysa = []
    entry_data = summary_data.loc[summary_data['c_a_station'] == (station, control_area)]

    xs = entry_data ['time_block']
    ys = entry_data ['entries']
    
    for i in range (0, 43):
        for j in range (0, len(xs)):
            if (xs[j] == i):
                xsa.append (i)
                ysa.append (ys[j])
    plt.plot (xsa, ysa)
    legends.append (station + ' at ' + control_area)


summary_data_sums = summary_data.groupby (['c_a_station']) ['entries'].apply (lambda x:x.sum())
#print (summary_data_sums)
print (summary_data_sums.sort_values(ascending=False))

legends = []

plt.savefig('top_five_control_areas.png', bbox_inches='tight')
axes= plt.axes()
axes.set_xticks ([0,6,12,18,24,30,36,42])
axes.set_xticklabels (['Saturday midnite','Sunday midnite','Monday midnite','Tuesday midnite','Wednesday midnite','Thursday midnite','Friday midnite','Saturday midnite'])



do_plotting ('GRD CNTRL-42 ST', 'R238A')
do_plotting ('34 ST-PENN STA', 'R138')
do_plotting ('FLUSHING-MAIN', 'R533')
do_plotting ('JKSN HT-ROOSVLT', 'N324')
do_plotting ('59 ST COLUMBUS', 'N051')
plt.legend (legends, shadow = True, loc = 0);
plt.title('Entries at Top Five Control Areas by Day of Week--2016 to 2019')
plt.xlabel ('Time of Week')
plt.ylabel ('Total Entries')
plt.axis ([0,42,0,1800000])
#plt.show()

plt.savefig('top_five_control_areas.png')
print('succeeded')


c_a_station
(GRD CNTRL-42 ST, R238)     20414637
(34 ST-PENN STA, R138)      19014550
(FLUSHING-MAIN, R533)       18201481
(JKSN HT-ROOSVLT, N324)     17040369
(59 ST COLUMBUS, N051)      16877656
(PATH NEW WTC, PTH22)       16685189
(86 ST, R250)               15313034
(14 ST-UNION SQ, R221)      14470702
(34 ST-HERALD SQ, N506)     14402053
(GRD CNTRL-42 ST, R240)     13738854
(W 4 ST-WASH SQ, N083)      12699433
(BEDFORD AV, H009)          12588033
(JAMAICA CENTER, N606)      11898349
(TIMES SQ-42 ST, R151)      11635452
(47-50 STS ROCK, N500)      11623595
(125 ST, R258)              11032882
(34 ST-HERALD SQ, A025)     11004409
(8 AV, H001)                10990899
(GRAND ST, N520)            10697190
(96 ST, R169)               10691248
(42 ST-PORT AUTH, N063A)    10670469
(68ST-HUNTER CO, R246)      10537599
(JOURNAL SQUARE, PTH03)     10507087
(SUTPHIN-ARCHER, N605)      10424235
(34 ST-PENN STA, N072)      10251311
(14 ST-UNION SQ, A033)      10031424
(THIRTY THIRD ST, PTH17)  

In [72]:
df2=df.drop(['linename','date','time','exits','division','desc'], axis=1)
print (len(df2.index))
df2=df2.drop(df2[df2.entry_increments < 1].index)
df2=df2.drop(df2[df2.station != 'GRD CNTRL-42 ST'].index)
print(df2)
df2.to_csv('/Users/skydivedavis/Documents/gct2.csv')

204934
          c/a  unit       scp          station  entries            datetime  \
157567   R236  R045  00-00-00  GRD CNTRL-42 ST  2022541 2019-06-08 09:00:00   
157568   R236  R045  00-00-00  GRD CNTRL-42 ST  2022671 2019-06-08 13:00:00   
157569   R236  R045  00-00-00  GRD CNTRL-42 ST  2022906 2019-06-08 17:00:00   
157570   R236  R045  00-00-00  GRD CNTRL-42 ST  2023080 2019-06-08 21:00:00   
157571   R236  R045  00-00-00  GRD CNTRL-42 ST  2023147 2019-06-09 05:00:00   
157572   R236  R045  00-00-00  GRD CNTRL-42 ST  2023167 2019-06-09 09:00:00   
157573   R236  R045  00-00-00  GRD CNTRL-42 ST  2023201 2019-06-09 13:00:00   
157574   R236  R045  00-00-00  GRD CNTRL-42 ST  2023308 2019-06-09 17:00:00   
157575   R236  R045  00-00-00  GRD CNTRL-42 ST  2023426 2019-06-09 21:00:00   
157576   R236  R045  00-00-00  GRD CNTRL-42 ST  2023467 2019-06-10 01:00:00   
157578   R236  R045  00-00-00  GRD CNTRL-42 ST  2023531 2019-06-10 09:00:00   
157579   R236  R045  00-00-00  GRD CNTRL-42 S

In [65]:
print(df2)
df2.to_csv('/Users/skydivedavis/Documents/gct.csv')

          c/a  unit       scp          station    entries            datetime  \
157648   R236  R045  00-00-02  GRD CNTRL-42 ST   10486054 2019-06-08 05:00:00   
157595   R236  R045  00-00-00  GRD CNTRL-42 ST    2029998 2019-06-13 05:00:00   
157746   R236  R045  00-00-04  GRD CNTRL-42 ST   12360879 2019-06-11 01:00:00   
158738   R238  R046  00-00-04  GRD CNTRL-42 ST     897296 2019-06-14 04:00:00   
157677   R236  R045  00-00-02  GRD CNTRL-42 ST   10491650 2019-06-13 05:00:00   
157659   R236  R045  00-00-02  GRD CNTRL-42 ST   10486547 2019-06-10 05:00:00   
157718   R236  R045  00-00-03  GRD CNTRL-42 ST    4008277 2019-06-13 05:00:00   
159082   R238  R046  00-05-00  GRD CNTRL-42 ST   34211474 2019-06-08 12:00:00   
159080   R238  R046  00-05-00  GRD CNTRL-42 ST   34211473 2019-06-08 04:00:00   
158919   R238  R046  00-03-01  GRD CNTRL-42 ST   21079908 2019-06-09 08:00:00   
158534   R238  R046  00-00-00  GRD CNTRL-42 ST      85724 2019-06-08 04:00:00   
158284   R237  R046  01-00-0

In [60]:
tuple1 = ('59 ST', 'A002', 1)

def string_from_tuple (tup):
    return tup[0]+','+tup[1]+','+str(tup[2])

summary_data = pd.DataFrame(
    data={'c_a_station_time_block': [tuple1],
          'c_a_station': [('59 ST', 'A002')],
          'time_block': [1],
          'entries': [7]},
    index=[string_from_tuple (tuple1)],
    columns=['c_a_station_time_block', 'c_a_station', 'time_block', 'entries'])

print (summary_data)

print (summary_data.loc [string_from_tuple (tuple1)])

summary_data.loc [string_from_tuple (('59 ST', 'A002', 2))] = [('59 ST', 'A002', 2), ('59 ST', 'A002'), 2, 5]
print (summary_data)

             c_a_station_time_block    c_a_station  time_block  entries
59 ST,A002,1       (59 ST, A002, 1)  (59 ST, A002)           1        7
c_a_station_time_block    (59 ST, A002, 1)
c_a_station                  (59 ST, A002)
time_block                               1
entries                                  7
Name: 59 ST,A002,1, dtype: object
             c_a_station_time_block    c_a_station  time_block  entries
59 ST,A002,1       (59 ST, A002, 1)  (59 ST, A002)           1        7
59 ST,A002,2       (59 ST, A002, 2)  (59 ST, A002)           2        5
